### MID 23
### Taking ant1.3 as training data.
### ant1.4 as testing data.
### Min max scaling done to few columns: ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
### Oversampling and smote methods used to increase size of training data.
### CNN model used, model type is 2 as per the BTP documentation spreadsheet.
### PCA with n_components = 15
### np.rint() used on predictions

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import Dense,Dropout,Conv2D,Conv1D,Flatten,MaxPool2D
import tensorflow as tf
import csv
from sklearn.decomposition import PCA

In [2]:
# Loading data. 
train_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv"
test_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"
train_data_name = "ant-1.3"
test_data_name = "ant-1.4"
ant_1_3 = pd.read_csv(train_data_path)
ant_1_4 = pd.read_csv(test_data_path)
files = ["/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv", "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"]
combined_data = pd.concat(map(pd.read_csv, files))

In [3]:
# Applying Min Max Scaling.
scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
combined_data[cols_to_norm] = MinMaxScaler().fit_transform(combined_data[cols_to_norm])

In [4]:
combined_data

name  version                                             name.1  \
0    ant      1.3            org.apache.tools.ant.taskdefs.ExecuteOn   
1    ant      1.3                 org.apache.tools.ant.DefaultLogger   
2    ant      1.3     org.apache.tools.ant.taskdefs.TaskOutputStream   
3    ant      1.3                  org.apache.tools.ant.taskdefs.Cvs   
4    ant      1.3             org.apache.tools.ant.taskdefs.Copyfile   
..   ...      ...                                                ...   
173  ant      1.4                   org.apache.tools.ant.TaskAdapter   
174  ant      1.4  org.apache.tools.ant.taskdefs.rmic.DefaultRmic...   
175  ant      1.4           org.apache.tools.ant.IntrospectionHelper   
176  ant      1.4  org.apache.tools.ant.taskdefs.compilers.Defaul...   
177  ant      1.4                org.apache.tools.ant.NoBannerLogger   

          wmc  dit    noc       cbo       rfc      lcom        ca  ...  \
0    0.142857  0.6  0.050  0.102941  0.214286  0.011856  0.014815  ...   
1    0.181818  0.0  0.025  0.058824  0.163265  0.020033  0.029630  ...   
2    0.038961  0.2  0.000  0.007353  0.045918  0.000000  0.000000  ...   
3    0.155844  0.4  0.000  0.088235  0.188776  0.013083  0.000000  ...   
4    0.077922  0.4  0.000  0.029412  0.107143  0.000409  0.000000  ...   
..        ...  ...    ...       ...       ...       ...       ...  ...   
173  0.064935  0.4  0.000  0.051471  0.096939  0.001635  0.029630  ...   
174  0.220779  0.0  0.075  0.110294  0.336735  0.036795  0.029630  ...   
175  0.298701  0.0  0.000  0.227941  0.418367  0.082175  0.162963  ...   
176  0.142857  0.0  0.200  0.161765  0.438776  0.006132  0.059259  ...   
177  0.051948  0.2  0.000  0.022059  0.081633  0.000000  0.000000  ...   

          dam       moa       mfa       cam    ic       cbm       amc  \
0    1.000000  0.111111  0.885057  0.232323  0.75  0.363636  0.165951   
1    1.000000  0.000000  0.000000  0.307692  0.00  0.000000  0.080979   
2    1.000000  0.111111  0.714286  0.666667  0.25  0.090909  0.083267   
3    1.000000  0.111111  0.770833  0.458333  0.00  0.000000  0.115693   
4    1.000000  0.000000  0.880952  0.416667  0.50  0.181818  0.100881   
..        ...       ...       ...       ...   ...       ...       ...   
173  0.000000  0.000000  0.902439  0.400000  0.25  0.090909  0.138351   
174  1.000000  0.222222  0.000000  0.197917  0.00  0.000000  0.172373   
175  0.444444  0.000000  0.000000  0.318182  0.00  0.000000  0.219724   
176  1.000000  0.777778  0.000000  0.266667  0.00  0.000000  0.373390   
177  1.000000  0.000000  0.842105  0.875000  0.25  0.090909  0.064852   

       max_cc    avg_cc  bug  
0    0.085714  0.209085    0  
1    0.171429  0.269903    2  
2    0.028571  0.109529    0  
3    0.085714  0.232742    0  
4    0.028571  0.136898    0  
..        ...       ...  ...  
173  0.028571  0.131428    0  
174  0.400000  0.396221    0  
175  0.685714  0.378561    0  
176  0.628571  0.761689    1  
177  0.142857  0.287498    0  

[303 rows x 24 columns]

In [5]:
# define transform
components = 15
pca = PCA(n_components=components)
# prepare transform on dataset
pca.fit(combined_data[cols_to_norm])
# apply transform to dataset
transformed = pca.transform(combined_data[cols_to_norm])

In [6]:
transformed = pd.DataFrame(transformed)
transformed 

0         1         2         3         4         5         6  \
0   -0.818904  0.255541  0.173010 -0.085265 -0.298426 -0.041536 -0.029952   
1    0.211850 -0.596714 -0.058649 -0.060459 -0.059547 -0.063873 -0.088019   
2   -0.310718 -0.021222 -0.538803 -0.015225 -0.015308  0.025091  0.105172   
3   -0.328366  0.034184 -0.105838 -0.033729  0.314725 -0.066018 -0.185431   
4   -0.516435  0.234332 -0.202375 -0.122822 -0.155654 -0.028101 -0.014558   
..        ...       ...       ...       ...       ...       ...       ...   
298  0.126544  0.701796  0.109947 -0.104515  0.057198 -0.305038  0.278453   
299  0.071521 -0.725060  0.254031  0.156662  0.066015 -0.159664 -0.159516   
300  0.276368 -0.485600  0.763979  0.403732  0.309357 -0.081984 -0.039603   
301 -0.023641 -0.851754  0.470180  0.584599  0.163868 -0.184260 -0.144292   
302 -0.305827  0.130311 -0.516061  0.225862  0.071176  0.149595  0.066592   

            7         8         9        10        11        12        13  \
0    0.060052 -0.061844  0.129277 -0.074484 -0.027763 -0.066784  0.002992   
1    0.048838  0.136331 -0.023472 -0.057784 -0.047066 -0.006698  0.003236   
2   -0.187430 -0.136162 -0.004927  0.121969 -0.112828 -0.021281  0.016337   
3    0.118595 -0.020596 -0.020034 -0.110471  0.004456  0.019378 -0.046558   
4   -0.023515  0.081734  0.039900  0.037578 -0.112383 -0.091509 -0.016293   
..        ...       ...       ...       ...       ...       ...       ...   
298 -0.261710 -0.178725 -0.005567 -0.002885 -0.048496 -0.053682  0.022157   
299  0.152804 -0.060837  0.065179 -0.091804 -0.021664  0.052310  0.057786   
300 -0.037146 -0.153829  0.234048 -0.577208 -0.073260  0.158787  0.093060   
301  0.435575 -0.433720  0.035253  0.218219  0.059088  0.027535  0.070731   
302 -0.080964 -0.008366 -0.034391  0.035003 -0.199711 -0.019391  0.001898   

           14  
0   -0.016871  
1    0.028059  
2   -0.073978  
3    0.056123  
4   -0.039552  
..        ...  
298  0.005590  
299 -0.071718  
300 -0.140269  
301 -0.130132  
302 -0.036359  

[303 rows x 15 columns]

In [7]:
train_data_index_list = list(range(ant_1_3.shape[0]))
train_data_index_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124]

In [8]:
test_data_index_list = list(range(ant_1_3.shape[0], ant_1_3.shape[0] + ant_1_4.shape[0]))
test_data_index_list

[125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291

In [9]:
X_train = transformed[transformed.index.isin(train_data_index_list)]
X_train = np.array(X_train)
Y_train = combined_data[transformed.index.isin(train_data_index_list)]
Y_train = Y_train['bug']

X_test = transformed[transformed.index.isin(test_data_index_list)]
X_test = np.array(X_test)
Y_test = combined_data[transformed.index.isin(test_data_index_list)]
Y_test = Y_test['bug']

In [10]:
# Applying over sampling and SMOTE to training data for augmentation.
ros = RandomOverSampler(random_state=0)
X_train, Y_train = ros.fit_resample(X_train, Y_train)

smt = SMOTE()
X_train, Y_train = smt.fit_resample(X_train, Y_train)


In [11]:
#Creating X_train dataframe after SMOTE
X_train = pd.DataFrame(X_train)
X_train

0         1         2         3         4         5         6  \
0   -0.818904  0.255541  0.173010 -0.085265 -0.298426 -0.041536 -0.029952   
1    0.211850 -0.596714 -0.058649 -0.060459 -0.059547 -0.063873 -0.088019   
2   -0.310718 -0.021222 -0.538803 -0.015225 -0.015308  0.025091  0.105172   
3   -0.328366  0.034184 -0.105838 -0.033729  0.314725 -0.066018 -0.185431   
4   -0.516435  0.234332 -0.202375 -0.122822 -0.155654 -0.028101 -0.014558   
..        ...       ...       ...       ...       ...       ...       ...   
415 -0.846266  0.119371  0.473178 -0.041280 -0.148751 -0.002356 -0.118953   
416 -0.662623  0.276089 -0.020254  0.102504 -0.065114 -0.057625 -0.039520   
417  0.133353 -0.654123  0.116196 -0.193381  0.026365 -0.092483 -0.130795   
418  0.133353 -0.654123  0.116196 -0.193381  0.026365 -0.092483 -0.130795   
419 -0.645816  0.164590 -0.085628 -0.091880 -0.181087 -0.070603  0.028691   

            7         8         9        10        11        12        13  \
0    0.060052 -0.061844  0.129277 -0.074484 -0.027763 -0.066784  0.002992   
1    0.048838  0.136331 -0.023472 -0.057784 -0.047066 -0.006698  0.003236   
2   -0.187430 -0.136162 -0.004927  0.121969 -0.112828 -0.021281  0.016337   
3    0.118595 -0.020596 -0.020034 -0.110471  0.004456  0.019378 -0.046558   
4   -0.023515  0.081734  0.039900  0.037578 -0.112383 -0.091509 -0.016293   
..        ...       ...       ...       ...       ...       ...       ...   
415 -0.141391  0.144937  0.249449 -0.080653  0.138782 -0.072715  0.085448   
416 -0.016481  0.126029  0.126982  0.045738  0.136236 -0.098762 -0.004033   
417  0.016852  0.051806  0.155039 -0.022490  0.059951  0.030020 -0.042572   
418  0.016852  0.051806  0.155039 -0.022490  0.059951  0.030020 -0.042572   
419 -0.019830 -0.083982  0.105403  0.026654 -0.119862  0.057715 -0.049209   

           14  
0   -0.016871  
1    0.028059  
2   -0.073978  
3    0.056123  
4   -0.039552  
..        ...  
415 -0.040622  
416  0.028861  
417 -0.007493  
418 -0.007493  
419  0.000580  

[420 rows x 15 columns]

In [12]:
#Creating Y_train dataframe after SMOTE
Y_train = pd.DataFrame(Y_train, columns=['bug'])
Y_train

bug
0      0
1      2
2      0
3      0
4      0
..   ...
415    3
416    3
417    3
418    3
419    3

[420 rows x 1 columns]

In [13]:
#Creating X_test dataframe after SMOTE
X_test = pd.DataFrame(X_test)
X_test 

0         1         2         3         4         5         6  \
0   -0.435557  0.006613 -0.001504 -0.037302  0.084933 -0.075094 -0.110340   
1   -0.585951  0.259560 -0.139178  0.010484 -0.100906  0.056142 -0.061287   
2    0.276324 -0.506007 -0.322392 -0.101244 -0.056399  0.169781 -0.051785   
3   -0.734317  0.341617 -0.030143  0.038302 -0.285238  0.064460  0.013935   
4    0.004661 -0.674220  0.266761 -0.169607 -0.127306  0.058459 -0.065243   
..        ...       ...       ...       ...       ...       ...       ...   
173  0.126544  0.701796  0.109947 -0.104515  0.057198 -0.305038  0.278453   
174  0.071521 -0.725060  0.254031  0.156662  0.066015 -0.159664 -0.159516   
175  0.276368 -0.485600  0.763979  0.403732  0.309357 -0.081984 -0.039603   
176 -0.023641 -0.851754  0.470180  0.584599  0.163868 -0.184260 -0.144292   
177 -0.305827  0.130311 -0.516061  0.225862  0.071176  0.149595  0.066592   

            7         8         9        10        11        12        13  \
0    0.081049 -0.045027 -0.046944  0.051590 -0.011918 -0.034569 -0.032980   
1   -0.063591  0.006644  0.213866 -0.011513 -0.052460  0.002295 -0.026511   
2   -0.050898  0.055162 -0.007508 -0.035479 -0.001263  0.006408  0.020929   
3   -0.005841  0.062722  0.031463 -0.052861 -0.032435 -0.130743  0.009916   
4   -0.217234  0.254638 -0.040460  0.068281 -0.066618 -0.052786  0.042342   
..        ...       ...       ...       ...       ...       ...       ...   
173 -0.261710 -0.178725 -0.005567 -0.002885 -0.048496 -0.053682  0.022157   
174  0.152804 -0.060837  0.065179 -0.091804 -0.021664  0.052310  0.057786   
175 -0.037146 -0.153829  0.234048 -0.577208 -0.073260  0.158787  0.093060   
176  0.435575 -0.433720  0.035253  0.218219  0.059088  0.027535  0.070731   
177 -0.080964 -0.008366 -0.034391  0.035003 -0.199711 -0.019391  0.001898   

           14  
0   -0.023919  
1    0.019942  
2    0.024118  
3   -0.067540  
4    0.058277  
..        ...  
173  0.005590  
174 -0.071718  
175 -0.140269  
176 -0.130132  
177 -0.036359  

[178 rows x 15 columns]

In [14]:
#Creating Y_test dataframe after SMOTE
Y_test = pd.DataFrame(Y_test, columns=['bug'])
Y_test

bug
0      0
1      0
2      0
3      0
4      0
..   ...
173    0
174    0
175    0
176    1
177    0

[178 rows x 1 columns]

In [15]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,components

In [16]:
# Preparing the data for the model.
X_train_matrix = X_train.values
X_test_matrix = X_test.values
Y_train_matrix = Y_train.values
Y_test_matrix = Y_test.values

In [17]:
# Preparing the data for the model.
Ytrainseries = Y_train['bug']
Ytestseries = Y_test['bug']

In [18]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [19]:
#Building the model
model = Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=1, activation='relu',input_shape=input_shape))
model.add(Conv2D(32, kernel_size=1, activation='relu'))
model.add(Conv2D(16, kernel_size=1, activation='relu'))
    
    
#model.add(MaxPool2D(pool_size=(1,8)))
#model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))

#compile model using mse as the loss function
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1, 15, 64)         128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 15, 32)         2080      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 15, 16)         528       
_________________________________________________________________
flatten (Flatten)            (None, 240)               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 1928      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 4,673
Trainable params: 4,673
Non-trainable params: 0
______________________________________________________

In [21]:
# Fitting the model on training data.
history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 128)

Epoch 1/100
4/4 [==============================] - 0s 2ms/step - loss: 3.4400 - mse: 3.4400 - mae: 1.4911 - root_mean_squared_error: 1.8547 - mean_squared_logarithmic_error: 0.8737
Epoch 2/100
4/4 [==============================] - 0s 2ms/step - loss: 3.2746 - mse: 3.2746 - mae: 1.4602 - root_mean_squared_error: 1.8096 - mean_squared_logarithmic_error: 0.7939
Epoch 3/100
4/4 [==============================] - 0s 2ms/step - loss: 3.0511 - mse: 3.0511 - mae: 1.4211 - root_mean_squared_error: 1.7467 - mean_squared_logarithmic_error: 0.6948
Epoch 4/100
4/4 [==============================] - 0s 2ms/step - loss: 2.7416 - mse: 2.7416 - mae: 1.3601 - root_mean_squared_error: 1.6558 - mean_squared_logarithmic_error: 0.5756
Epoch 5/100
4/4 [==============================] - 0s 2ms/step - loss: 2.3482 - mse: 2.3482 - mae: 1.2705 - root_mean_squared_error: 1.5324 - mean_squared_logarithmic_error: 0.4500
Epoch 6/100
4/4 [==============================] - 0s 2ms/step - loss: 1.8960 - mse: 1.8960 - m

4/4 [==============================] - 0s 2ms/step - loss: 0.3390 - mse: 0.3390 - mae: 0.4351 - root_mean_squared_error: 0.5822 - mean_squared_logarithmic_error: 0.1006
Epoch 47/100
4/4 [==============================] - 0s 2ms/step - loss: 0.3183 - mse: 0.3183 - mae: 0.4115 - root_mean_squared_error: 0.5642 - mean_squared_logarithmic_error: 0.0958
Epoch 48/100
4/4 [==============================] - 0s 2ms/step - loss: 0.2992 - mse: 0.2992 - mae: 0.3937 - root_mean_squared_error: 0.5470 - mean_squared_logarithmic_error: 0.0908
Epoch 49/100
4/4 [==============================] - 0s 2ms/step - loss: 0.2840 - mse: 0.2840 - mae: 0.3824 - root_mean_squared_error: 0.5329 - mean_squared_logarithmic_error: 0.0874
Epoch 50/100
4/4 [==============================] - 0s 2ms/step - loss: 0.2715 - mse: 0.2715 - mae: 0.3727 - root_mean_squared_error: 0.5210 - mean_squared_logarithmic_error: 0.0834
Epoch 51/100
4/4 [==============================] - 0s 2ms/step - loss: 0.2581 - mse: 0.2581 - mae: 0.3

4/4 [==============================] - 0s 1ms/step - loss: 0.1217 - mse: 0.1217 - mae: 0.1865 - root_mean_squared_error: 0.3489 - mean_squared_logarithmic_error: 0.0463
Epoch 92/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1201 - mse: 0.1201 - mae: 0.1971 - root_mean_squared_error: 0.3466 - mean_squared_logarithmic_error: 0.0452
Epoch 93/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1193 - mse: 0.1193 - mae: 0.1882 - root_mean_squared_error: 0.3453 - mean_squared_logarithmic_error: 0.0455
Epoch 94/100
4/4 [==============================] - 0s 1ms/step - loss: 0.1206 - mse: 0.1206 - mae: 0.1972 - root_mean_squared_error: 0.3472 - mean_squared_logarithmic_error: 0.0451
Epoch 95/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1203 - mse: 0.1203 - mae: 0.1888 - root_mean_squared_error: 0.3469 - mean_squared_logarithmic_error: 0.0459
Epoch 96/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1174 - mse: 0.1174 - mae: 0.1

In [22]:
# Getting the predictions.
predictions_y = model.predict(X_test1)
predictions_y

array([[0.65852314],
       [1.6688284 ],
       [0.        ],
       [0.4770813 ],
       [0.        ],
       [0.4434572 ],
       [0.855782  ],
       [0.        ],
       [0.32262728],
       [0.94404256],
       [0.7940975 ],
       [1.2286109 ],
       [0.02611209],
       [1.0323598 ],
       [1.5911694 ],
       [0.50479406],
       [1.0691545 ],
       [0.18915305],
       [0.02923393],
       [0.07693546],
       [0.        ],
       [0.01139685],
       [0.        ],
       [0.8388224 ],
       [3.021534  ],
       [1.030035  ],
       [0.        ],
       [0.        ],
       [0.21672475],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.273483  ],
       [0.02923393],
       [0.        ],
       [1.0847125 ],
       [0.02923393],
       [0.39541757],
       [0.        ],
       [1.9048572 ],
       [0.        ],
       [0.        ],
       [0.02923393],
       [0.        ],
       [0.02923393],
       [0.1678642 ],
       [2.643

In [23]:
# Rounding off the predictions to nearest
#integer as count of bugs is an integer.
predictions_y_round = np.rint(predictions_y)
predictions_y_round 

array([[1.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [2.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [3.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [3.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [2.],
       [0.],
       [0.],
       [0.],
       [1.],
       [3.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [3.],
       [1.],

In [24]:
# Viewing the number of predictions.
predictions_y_round.shape

(178, 1)

In [25]:
# Getting the sum of all the predictions obtained to used while obtaining FPA
s = 0
for  t in range(predictions_y_round.shape[0]):
    s+=predictions_y_round[t]
s

array([83.], dtype=float32)

In [26]:
# Obtaining the value of FPA metric for the model
Fpa = 0
for  t in range(predictions_y_round.shape[0]):
        x = 0
        for j in range( predictions_y_round.shape[0]-t+1, predictions_y_round.shape[0]):
               x = x + predictions_y_round[j]
        
        x = (x/s)/predictions_y_round.shape[0]
        Fpa = Fpa + x
Fpa

array([0.47874647], dtype=float32)

In [27]:
# Obtaining the value of CLC metric for the model
previous_obtained = predictions_y_round[predictions_y_round.shape[0] - 1]/s

CLC = 0
for i in range(predictions_y_round.shape[0]):
    if(i==0):
        CLC += 0 + previous_obtained
    else:
        additional = (predictions_y_round[predictions_y_round.shape[0] - 1 - i])/s
        CLC += 2*previous_obtained + additional
        previous_obtained += additional
        
CLC/=(2*predictions_y_round.shape[0])
CLC

array([0.48710555], dtype=float32)

In [28]:
# Getting direct metric results using the metrics given to model.
score = model.evaluate(X_test1, Y_test_matrix)
print(history.history.keys())
model.test_on_batch(X_test1, Y_test_matrix)
model.metrics_names
#print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[3], score[3]*100))


6/6 [==============================] - 0s 672us/step - loss: 0.7430 - mse: 0.7430 - mae: 0.5382 - root_mean_squared_error: 0.8619 - mean_squared_logarithmic_error: 0.2481
dict_keys(['loss', 'mse', 'mae', 'root_mean_squared_error', 'mean_squared_logarithmic_error'])


['loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error']

In [29]:
# Saving the model
model_id = 23
path_to_save = '/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model2/ant1.3_ant1.4_model2_4'
model.save(path_to_save)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model2/ant1.3_ant1.4_model2_4/assets


In [30]:
# Writing the results to csv file.
heading = ['model_id', 'train_data_name', 'test_data_name'] + model.metrics_names + ['fpa', 'clc']
score = [model_id, train_data_name, test_data_name] + score + [float(Fpa) , float(CLC)]

In [31]:
heading

['model_id',
 'train_data_name',
 'test_data_name',
 'loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error',
 'fpa',
 'clc']

In [32]:
score

[23,
 'ant-1.3',
 'ant-1.4',
 0.7429512143135071,
 0.7429512143135071,
 0.5381984114646912,
 0.861946165561676,
 0.24811361730098724,
 0.4787464737892151,
 0.4871055483818054]

In [33]:
# Writing the results to csv file.
with open(path_to_save + '_metric_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(heading)
    writer.writerow(score)


In [34]:
# Loading the model and to make sure that the model is saved properly.
model_loaded = load_model(path_to_save)